<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [3]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [5]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    



To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [ ]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [6]:
import requests


static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"


response = requests.get(static_url)


if response.status_code == 200:
    print("Página HTML descargada exitosamente.")
    html_content = response.text  
else:
    print(f"Error al obtener la página: {response.status_code}")


Página HTML descargada exitosamente.


import requests

# URL de la página de Wikipedia con la instantánea de los lanzamientos de Falcon 9
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Solicitar la página HTML
response = requests.get(static_url)

# Verificar si la solicitud fue exitosa (código 200)
if response.status_code == 200:
    print("Página HTML descargada exitosamente.")
    html_content = response.text  # Contenido de la página HTML
else:
    print(f"Error al obtener la página: {response.status_code}")
Create a `BeautifulSoup` object from the HTML `response`


In [7]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


Print the page title to verify if the `BeautifulSoup` object was created properly 


In [8]:
from bs4 import BeautifulSoup
import requests


static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"


response = requests.get(static_url)


if response.status_code == 200:
    print("Página HTML descargada exitosamente.")
    html_content = response.text  
    
    
    soup = BeautifulSoup(html_content, 'html.parser')
    
    
    print("Título de la página:", soup.title.string)
else:
    print(f"Error al obtener la página: {response.status_code}")

    

Página HTML descargada exitosamente.
Título de la página: List of Falcon 9 and Falcon Heavy launches - Wikipedia


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [9]:
from bs4 import BeautifulSoup
import requests


static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"


response = requests.get(static_url)


if response.status_code == 200:
    print("Página HTML descargada exitosamente.")
    html_content = response.text 
    
   
    soup = BeautifulSoup(html_content, 'html.parser')
    
    
    tables = soup.find_all('table')
    print(f"Se encontraron {len(tables)} tablas en la página.")

   
    for i, table in enumerate(tables):
        headers = table.find_all('th')
        column_names = [header.get_text(strip=True) for header in headers]
        if column_names:  # Solo mostrar si hay columnas
            print(f"Columnas en la tabla {i+1}: {column_names}")
else:
    print(f"Error al obtener la página: {response.status_code}")


Página HTML descargada exitosamente.
Se encontraron 26 tablas en la página.
Columnas en la tabla 3: ['Flight No.', 'Date andtime (UTC)', 'Version,Booster[b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '1', '2', '3', '4', '5', '6', '7']
Columnas en la tabla 4: ['Flight No.', 'Date andtime (UTC)', 'Version,Booster[b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '8', '9', '10', '11', '12', '13']
Columnas en la tabla 5: ['Flight No.', 'Date andtime (UTC)', 'Version,Booster[b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '14', '15', '16', '17', '18', '19', '20']
Columnas en la tabla 6: ['Flight No.', 'Date andtime (UTC)', 'Version,Booster[b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '21', '22', '23', '24', '25', '26', '27', '28', 'N/A[e]']
Columnas en la tabl

Starting from the third table is our target table contains the actual launch records.


In [10]:
from bs4 import BeautifulSoup
import requests


static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"


response = requests.get(static_url)


if response.status_code == 200:
    print("Página HTML descargada exitosamente.")
    html_content = response.text  
    
    
    soup = BeautifulSoup(html_content, 'html.parser')
    
    
    tables = soup.find_all('table')
   
    if len(tables) >= 3:
        print(f"Se encontraron {len(tables)} tablas en la página.")

        target_table = tables[2]

        headers = target_table.find_all('th')
        column_names = [header.get_text(strip=True) for header in headers]
        
        if column_names:
            print(f"Columnas en la tercera tabla: {column_names}")
        else:
            print("No se encontraron encabezados en la tercera tabla.")
    else:
        print("No hay suficientes tablas en la página.")
else:
    print(f"Error al obtener la página: {response.status_code}")

    

Página HTML descargada exitosamente.
Se encontraron 26 tablas en la página.
Columnas en la tercera tabla: ['Flight No.', 'Date andtime (UTC)', 'Version,Booster[b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '1', '2', '3', '4', '5', '6', '7']


You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [11]:
from bs4 import BeautifulSoup
import requests


def extract_column_from_header(row):
    """
    Esta función extrae el nombre de la columna de un encabezado de tabla,
    eliminando etiquetas innecesarias como <br>, <a> y <sup>.
    """
    if row.br:
        row.br.extract()  
    if row.a:
        row.a.extract()  
    if row.sup:
        row.sup.extract()  
        
    column_name = ' '.join(row.contents)  
    
   
    if not column_name.strip().isdigit():
        column_name = column_name.strip()
        return column_name
    return None


static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"


response = requests.get(static_url)


if response.status_code == 200:
    print("Página HTML descargada exitosamente.")
    html_content = response.text 
    
    
    soup = BeautifulSoup(html_content, 'html.parser')
    
    
    tables = soup.find_all('table')
    
   
    if len(tables) >= 3:
        print(f"Se encontraron {len(tables)} tablas en la página.")
        
        
        target_table = tables[2]
        
        
        headers = target_table.find_all('th')
        
        
        column_names = []
        for header in headers:
            column_name = extract_column_from_header(header)
            if column_name:  # Solo agregar si el nombre de la columna no es None
                column_names.append(column_name)
        
        
        if column_names:
            print(f"Columnas extraídas de la tercera tabla: {column_names}")
        else:
            print("No se encontraron nombres de columnas válidos.")
    else:
        print("No hay suficientes tablas en la página.")
else:
    print(f"Error al obtener la página: {response.status_code}")


Página HTML descargada exitosamente.
Se encontraron 26 tablas en la página.
Columnas extraídas de la tercera tabla: ['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


Check the extracted column names


In [12]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [15]:

launch_dict = dict.fromkeys(column_names)


del launch_dict['Date and time ( )']


launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []


launch_dict['Version Booster'] = []
launch_dict['Booster landing'] = []
launch_dict['Date'] = []
launch_dict['Time'] = []


rows = target_table.find_all('tr')[1:] 


for row in rows:
    cells = row.find_all('td')
    if len(cells) == len(column_names):  
        for idx, cell in enumerate(cells):
            cell_text = cell.get_text(strip=True)

            
            column_name = column_names[idx]
            if column_name in launch_dict:
                launch_dict[column_name].append(cell_text)


print(f"Datos extraídos: {launch_dict}")


Datos extraídos: {'Flight No.': [], 'Launch site': [], 'Payload': [], 'Payload mass': [], 'Orbit': [], 'Customer': [], 'Launch outcome': [], 'Version Booster': [], 'Booster landing': [], 'Date': [], 'Time': []}


Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [19]:
import re


def clean_text(text):
   
    text = re.sub(r'\[.*?\]', '', text)
    
   
    if text in ["N/A", "–"]:
        return ""
    
   
    return text.strip()


for row in rows:
    cells = row.find_all('td')
    if len(cells) == len(column_names):  
        for idx, cell in enumerate(cells):
            cell_text = clean_text(cell.get_text())  
            
            column_name = column_names[idx]
            
            
            if column_name in launch_dict:
                launch_dict[column_name].append(cell_text)


print(f"Datos extraídos: {launch_dict}")


Datos extraídos: {'Flight No.': [], 'Launch site': [], 'Payload': [], 'Payload mass': [], 'Orbit': [], 'Customer': [], 'Launch outcome': [], 'Version Booster': [], 'Booster landing': [], 'Date': [], 'Time': []}


After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [16]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
